In [14]:
import networkx as nx
import numpy as np
import pandas as pd 

from ete3 import Tree
import utilities.validate_trees as tree_val

import sys
import os

from tqdm import tqdm_notebook

import seaborn as sns
import colorcet as cc
import matplotlib.pyplot as plt

import cassiopeia.TreeSolver.compute_meta_purity as cmp 

import seaborn as sns
sns.set_context("talk")
sns.set_style("whitegrid")

In [3]:
meta = pd.read_csv("../JQ19/5k/RNA/LG_meta.txt", sep='\t', index_col = 0)

In [8]:
labels = []
tree_fps = []
lg_to_tree = {}
i = 0
for tree_dir in tqdm_notebook(os.listdir("/data/yosef2/users/mattjones/projects/metastasis/JQ19/5k/trees")):
    tree_dir = "/data/yosef2/users/mattjones/projects/metastasis/JQ19/5k/trees/" + tree_dir + "/"
    if not os.path.isdir(tree_dir):
        continue
    
    lg = tree_dir.split("/")[-2][2:]
    fps = [fp for fp in os.listdir(tree_dir) if ".processed.pkl" in fp and 'nj' not in fp]
    cmfp = [fp for fp in os.listdir(tree_dir) if "_character_matrix.txt" in fp]
    if len(fps) > 0:
#             tree = nx.read_gpickle(tree_dir + fps[0]).get_network()
#             for e in tree.edges():
#                 print(e[0].get_mut_length(e[1]))
            try:
                tree = nx.read_gpickle(tree_dir + fps[0]).get_network()
                lg_to_tree[int(lg)] = tree_val.assign_edge_lengths(tree)
                tree_fps.append(tree_dir + fps[0])
                cmfp = tree_dir + cmfp[0]
                labels.append(int(lg))
                cm = pd.read_csv(cmfp, sep='\t', index_col = 0, dtype=str)
            except:
                i += 1

assigning edge lengths: 100%|██████████| 97/97 [00:00<00:00, 94439.99it/s]


In [13]:
cm = pd.read_csv("/data/yosef2/users/mattjones/projects/metastasis/JQ19/5k/trees/lg3/lg3_character_matrix.txt", sep='\t', index_col = 0)
print('number of cells: ', cm.shape[0])
print('number of unique cells', cm.drop_duplicates().shape[0])

lg3 = lg_to_tree[3]
root = [n for n in lg3 if lg3.in_degree(n) == 0][0]
lg3 = cmp.set_depth(lg3, root)

tree_depths = [lg3.nodes[n]['depth'] for n in lg3 if lg3.out_degree(n) == 0]
print('avg tree depth', np.mean(tree_depths))

sns.dens

number of cells:  5616
number of unique cells 5560
avg tree depth 15.544159544159545


In [ ]:
tree_dists, edit_dists, all_pairs, diam, n_targets = tree_val.compute_pairwise_dist_nx(lg_to_tree[19])
print(tree_dists)
hist = plt.hist2d(tree_dists,edit_dists, cmap=cc.cm.CET_L19)
xlab = plt.xlabel("Phylogenetic Distance")
ylab = plt.ylabel("Allele Distance")


plt.show()

In [ ]:
all_tree_dists = []
all_edit_dists = []

max_diam, max_targets = 0,0
for tree in tqdm_notebook(lg_to_tree.values()):
    
    tree_dists, edit_dists,all_pairs,diam,n_targets = tree_val.compute_pairwise_dist_nx(tree)
    #tree_dists /= (n_targets)**2
    #edit_dists /= (n_targets)**2
    
    all_tree_dists += list(tree_dists)
    all_edit_dists += list(edit_dists)
    
    max_diam = max(max_diam, diam)
    max_targets = max(max_targets, n_targets)



In [ ]:
tree_val.dist_plotter(all_tree_dists, all_edit_dists, '2D-Hist', diam=20, n_targets = 20, out_fp = 'all_phy_vs_edit.pdf')

In [ ]:
intra_dists = []
intra_edit_dists = []

max_diam, max_targets = 0,0
for n in tqdm_notebook(lg_to_tree.keys()):
    
    tree = lg_to_tree[n]
    if n in [1,2,3]:
        continue
        
    tree_dists, edit_dists,all_pairs,diam,n_targets = tree_val.compute_pairwise_dist_nx(tree, compare_method = "intra", meta_item = meta['sampleID'])
    #tree_dists /= (n_targets)**2
    #edit_dists /= (n_targets)**2
    
    intra_dists += list(tree_dists)
    intra_edit_dists += list(edit_dists)
    
    max_diam = max(max_diam, diam)
    max_targets = max(max_targets, n_targets)

In [ ]:
tree_val.dist_plotter(intra_dists, intra_edit_dists, '2D-Hist', diam=20, n_targets = 20, out_fp = 'intra_phy_vs_edit.pdf')